In [1]:
import requests
from bs4 import BeautifulSoup
import time
import os  # 파일 경로 처리를 위한 os 모듈 추가

def crawl_and_save_urls_advanced(url_file_path, output_dir_path):
    """
    URL 파일에서 URL을 읽어 크롤링하고, 각 URL의 내용을 개별 파일에 저장합니다.
    오류 처리, robots.txt 준수 (간단한 확인), User-Agent 설정, 파일명 안전 처리 등을 포함합니다.

    Args:
        url_file_path (str): URL이 포함된 파일 경로.
        output_dir_path (str): 크롤링 결과를 저장할 디렉토리 경로.
    """

    try:
        with open(url_file_path, 'r', encoding='utf-8') as url_file:
            urls = [line.strip() for line in url_file.readlines()]
    except FileNotFoundError:
        print(f"오류: '{url_file_path}' 파일을 찾을 수 없습니다.")
        return

    # 출력 디렉토리가 없으면 생성
    if not os.path.exists(output_dir_path):
        os.makedirs(output_dir_path)

    # User-Agent 설정 (웹사이트 요청 시 '사람'처럼 보이도록 설정)
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36'
    }

    for url in urls:
        try:
            # robots.txt 확인 (간단한 예시, 실제 robots.txt 파싱 라이브러리 사용하는 것이 좋음)
            base_url = "/".join(url.split("/")[:3])  # URL에서 기본 주소 추출 (예: https://www.example.com)
            robots_url = base_url + "/robots.txt"
            try:
                robots_response = requests.get(robots_url, headers=headers)
                if robots_response.status_code == 200:
                    # 간단한 robots.txt 확인: "Disallow" 규칙 포함 여부 확인 (완벽하지 않음)
                    if "Disallow" in robots_response.text:
                        print(f"경고: robots.txt 규칙에 따라 크롤링을 제한할 수 있습니다: {url}")
                        # robots.txt를 더 엄격하게 확인하고 준수하는 것이 좋습니다.
                else:
                    print(f"경고: robots.txt를 가져올 수 없습니다: {robots_url}")
            except requests.exceptions.RequestException as e:
                print(f"경고: robots.txt 확인 중 오류 발생: {robots_url} - {e}")

            response = requests.get(url, headers=headers)
            response.raise_for_status()  # 오류 발생 시 예외 발생

            soup = BeautifulSoup(response.text, 'html.parser')

            # 파일명 안전하게 만들기 (URL에서 유효하지 않은 문자 제거)
            file_name = url.replace("https://", "").replace("http://", "").replace("/", "_").replace("?", "_").replace("%", "_").replace("*", "_") + ".txt"
            file_path = os.path.join(output_dir_path, file_name)

            with open(file_path, 'w', encoding='utf-8') as output_file:
                # 여기에서 soup를 사용하여 원하는 데이터를 추출합니다.
                # 예시로, 페이지의 모든 텍스트를 가져와 저장합니다.
                page_text = soup.get_text()
                output_file.write(f"URL: {url}\n")
                output_file.write(page_text)

            print(f"성공적으로 크롤링 및 저장: {url} -> {file_path}")
            time.sleep(1)  # 1초 지연 (크롤링 예절)

        except requests.exceptions.RequestException as e:
            print(f"오류: {url} - {e}")
        except Exception as e:
            print(f"예상치 못한 오류: {url} - {e}")

# 실행 예시
url_file_path = "sorted_urls.txt"  # 제공된 파일 이름으로 변경
output_dir_path = "crawled_data"  # 결과를 저장할 디렉토리 이름
crawl_and_save_urls_advanced(url_file_path, output_dir_path)

경고: robots.txt 규칙에 따라 크롤링을 제한할 수 있습니다: https://jaranda.kr/solution/312-%EC%95%84%EC%9D%B4%EA%B0%80-%EC%A7%91%EC%95%88%EC%9D%BC%EC%9D%84-%EC%9A%94%EC%B2%AD%ED%95%B4%EB%8F%84-%EB%8F%84%EC%99%80%EC%A3%BC%EC%A7%80-%EC%95%8A%EC%95%84%EC%9A%94
예상치 못한 오류: https://jaranda.kr/solution/312-%EC%95%84%EC%9D%B4%EA%B0%80-%EC%A7%91%EC%95%88%EC%9D%BC%EC%9D%84-%EC%9A%94%EC%B2%AD%ED%95%B4%EB%8F%84-%EB%8F%84%EC%99%80%EC%A3%BC%EC%A7%80-%EC%95%8A%EC%95%84%EC%9A%94 - [Errno 2] No such file or directory: 'crawled_data\\jaranda.kr_solution_312-_EC_95_84_EC_9D_B4_EA_B0_80-_EC_A7_91_EC_95_88_EC_9D_BC_EC_9D_84-_EC_9A_94_EC_B2_AD_ED_95_B4_EB_8F_84-_EB_8F_84_EC_99_80_EC_A3_BC_EC_A7_80-_EC_95_8A_EC_95_84_EC_9A_94.txt'
경고: robots.txt 규칙에 따라 크롤링을 제한할 수 있습니다: https://www.chaisplay.com/stories/3951-%EC%95%84%EC%9D%B4%EB%93%A4%EA%B3%BC-%EC%A7%91%EC%95%88%EC%9D%BC%EC%9D%84-%EA%B0%99%EC%9D%B4-%ED%95%B4%EC%95%BC-%ED%95%98%EB%8A%94-%EC%A4%91%EC%9A%94%ED%95%9C-%EC%9D%B4%EC%9C%A0
예상치 못한 오류: https://www.chaisplay.com/stories/3